```{contents}
```
## LLM Response Caching 

**LLM response caching** is the practice of storing the **final output produced by a language model** for a given prompt and reusing it when the same (or equivalent) prompt appears again.

It prevents repeated model calls for identical work.

**Resulting effects**

* Lower latency
* Lower API / compute cost
* Higher request capacity
* Increased reliability during traffic spikes

---

### Position in the Request Pipeline

```
Client Request
      ↓
Cache Lookup ── hit → Return Cached Response
      ↓ miss
Prompt Builder → LLM → Response → Store in Cache → Return
```

**Demonstration**

Only when the cache misses does the system execute the LLM.

---

### Core Cache Strategy: Exact Match

#### Concept

The **same normalized prompt** should always return the **same stored output**.

#### Demonstration (Python)

```python
response_cache = {}

def get_llm_response(prompt: str):
    key = prompt.strip().lower()

    if key in response_cache:
        print("Cache Hit")
        return response_cache[key]

    print("Cache Miss — Calling LLM")
    response = llm.invoke(prompt).content
    response_cache[key] = response
    return response
```

---

### Semantic Response Caching (Similarity-Based)

When prompts are not identical but **meaningfully similar**, embeddings allow reuse.

#### Demonstration

```python
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")
semantic_cache = []

def get_semantic_response(prompt):
    q_vec = model.encode(prompt)

    for cached_prompt, vec, response in semantic_cache:
        similarity = np.dot(q_vec, vec)
        if similarity > 0.9:
            print("Semantic Cache Hit")
            return response

    print("Semantic Cache Miss — Calling LLM")
    response = llm.invoke(prompt).content
    semantic_cache.append((prompt, q_vec, response))
    return response
```

---

### Cache Invalidation (Critical)

Caching is only safe if **invalidated correctly**.

#### Demonstration

```python
def invalidate_response_cache(key):
    response_cache.pop(key, None)
```

**Invalidate when**

* Prompt templates change
* Model version changes
* Knowledge base changes
* Business rules change

---

### TTL-Based Production Cache

Prevents stale content.

#### Demonstration

```python
from cachetools import TTLCache

response_cache = TTLCache(maxsize=10000, ttl=3600)
```

Responses expire automatically after one hour.

---

### 7. What Should Be Cached

| Layer        | Cache Content     |
| ------------ | ----------------- |
| Final output | LLM response text |
| Retrieval    | RAG documents     |
| Prompts      | Compiled prompt   |
| Embeddings   | Query embeddings  |

---

### Production Best Practices

* Normalize prompt before hashing
* Include **model name + version** in cache key
* Apply TTL and eviction policy
* Use Redis or Memcached for distributed systems
* Track cache hit ratio

---

### Mental Model

```
LLM Response Caching = Short-term memory for your AI system
```

---

### Key Takeaways

* Response caching is one of the highest impact optimizations
* Eliminates redundant LLM calls
* Reduces cost and latency dramatically
* Essential for scalable LLM architectures